# Centrality computation example for inner London

In [1]:
%matplotlib inline
%pip install cityseer

Note: you may need to restart the kernel to use updated packages.


## Data and graph cleaning

This example will make use of OSM data downloaded from the [OSM API](https://wiki.openstreetmap.org/wiki/API) for a 5000m radius around London Soho.

For additional information on network preparation, see the corresponding notebook on [graph cleaning](https://colab.research.google.com/github/cityseer/cityseer/blob/master/demos/graph_cleaning.ipynb).

In [2]:
import utm
from shapely import geometry

from cityseer.tools import graphs, mock

# download from OSM - see docs if you want examples for how to use osmNx for this step
lng, lat = -0.13396079424572427, 51.51371088849723
G_utm = mock.make_buffered_osm_graph(lng, lat, 5000)

INFO:cityseer.tools.graphs:Converting networkX graph from WGS to UTM.
INFO:cityseer.tools.graphs:Processing node x, y coordinates.
100%|██████████| 142110/142110 [00:03<00:00, 43819.92it/s]
INFO:cityseer.tools.graphs:Processing edge geom coordinates, if present.
100%|██████████| 159914/159914 [00:00<00:00, 1953077.80it/s]


<Figure size 432x288 with 0 Axes>

In [3]:
# simplify - this is not necessary if using "clean" graph sources - e.g. OS Open Roads
G = graphs.nx_simple_geoms(G_utm)
G = graphs.nx_remove_filler_nodes(G)
G = graphs.nx_remove_dangling_nodes(G, despine=20, remove_disconnected=True)
G = graphs.nx_remove_filler_nodes(G)
G1 = graphs.nx_consolidate_nodes(G, buffer_dist=10, min_node_group=3)
G2 = graphs.nx_split_opposing_geoms(G1, buffer_dist=15)
G3 = graphs.nx_consolidate_nodes(G2, buffer_dist=15, crawl=False, min_node_degree=2, cent_min_degree=4)

INFO:cityseer.tools.graphs:Generating simple (straight) edge geometries.
100%|██████████| 159914/159914 [00:00<00:00, 175136.90it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 142110/142110 [00:06<00:00, 21335.83it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████| 45773/45773 [00:00<00:00, 806101.11it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 41493/41493 [00:00<00:00, 134080.64it/s]
INFO:cityseer.tools.graphs:Consolidating nodes.
/Users/gareth/dev/cityseer-api/cityseer/tools/graphs.py:870: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  point_geom.nd_key = nd_key
/Users/gareth/dev/cityseer-api/cityseer/tools/graphs.py:871: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  point_geom.degree = nx.degree(nx_multigraph, nd_key)
100%|█████████

In [4]:
from cityseer.metrics import networks

# decompose for higher resolution analysis
G_decomp = graphs.nx_decompose(G3, 25)
# create a cityseer network layer
N = networks.NetworkLayerFromNX(G_decomp, distances=[250, 500, 2000])
# calculate centralities
# it is quicker to run this analysis on a machine with multiple cores due to the area and resolution of the analysis
# e.g. 8 core 4.2GHz machine ~ 10 minutes
# if you want to compute wider area centralities, e.g. 20km, then use less decomposition to speed up the computation
N.node_centrality(measures=["node_beta", "node_betweenness"])

INFO:cityseer.tools.graphs:Decomposing graph to maximum edge lengths of 25.
100%|██████████| 35477/35477 [00:07<00:00, 4545.98it/s]
INFO:cityseer.tools.graphs:Preparing node and edge arrays from networkX graph.
100%|██████████| 90445/90445 [00:07<00:00, 11432.87it/s]
INFO:cityseer.metrics.networks:Computing node_beta, node_betweenness centrality measures using shortest (non-angular) path heuristic.


  0%|          | 0/90445 [00:00<?, ?it/s]

/Users/gareth/dev/cityseer-api/cityseer/algos/centrality.py:321: NumbaWarning: Cannot cache compiled function "shortest_path_tree" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  tree_map = shortest_path_tree(
/Users/gareth/dev/cityseer-api/cityseer/algos/centrality.py:321: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  tree_map = shortest_path_tree(
/Users/gareth/dev/cityseer-api/cityseer/algos/centrality.py:321: NumbaWarning: Cannot cache compiled function "shortest_path_tree" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  tree_map = shortest_path_tree(
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/gareth/dev/cityseer-api/cityseer/algos/centrality.py:237: NumbaWarning: Cannot cache compiled function "local_node_centrality" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @njit(cache=True, fastmath=co

: 

In [ ]:
# data can be extracted, plotted, or otherwise manipulated directly from the numpy arrays
# see the getting started guide if you'd prefer to first convert back to NetworkX
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import minmax_scale


def simple_plot(xs: np.ndarray, ys: np.ndarray, vals: np.ndarray):
    """ """
    # extents
    easting, northing = utm.from_latlon(lat, lng)[:2]
    buff = geometry.Point(easting, northing).buffer(3500)
    min_x, min_y, max_x, max_y = buff.bounds
    # fig
    fig, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=200, facecolor="white")
    # filter
    select = xs > min_x
    select = np.logical_and(select, xs < max_x)
    select = np.logical_and(select, ys > min_y)
    select = np.logical_and(select, ys < max_y)
    select_idx = np.where(select)[0]
    # remove any extreme outliers
    v = np.clip(vals, np.nanpercentile(vals, 0.01), np.nanpercentile(vals, 99.9))
    # shape if wanted
    c = v**1
    c = minmax_scale(c, feature_range=(0, 1))
    s = v**1.5
    s = minmax_scale(s, feature_range=(0, 1.5))
    # plot
    ax.scatter(
        xs[select_idx], ys[select_idx], c=c[select_idx], s=s[select_idx], linewidths=0, edgecolors="none", cmap="Reds"
    )
    # limits
    ax.set_xlim(left=min_x, right=max_x)
    ax.set_ylim(bottom=min_y, top=max_y)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1)
    ax.set_facecolor("white")
    plt.show()

In [ ]:
for d in [250, 500, 2000]:
    b = networks.beta_from_distance(d)[0]
    avg_d = networks.avg_distance_for_beta(b)[0]
    print(
        f"""
    "Gravity" index / spatial impedance weighted (closeness-like) centrality:
    Avg walking tolerance: {avg_d:.2f}m
    Beta: {b} (spatial impedance factor)
    Max walking tolerance: {d}m
    """
    )
    simple_plot(N.node_x_arr, N.node_y_arr, N.metrics["centrality"]["node_beta"][d])

for d in [250, 500, 2000]:
    print(
        f"""
    Spatial impedance weighted betweenness centrality:
    Avg walking tolerance: {avg_d:.2f}m
    Beta: {b} (spatial impedance factor)
    Max walking tolerance: {d}m
    """
    )
    simple_plot(N.node_x_arr, N.node_y_arr, N.metrics["centrality"]["node_betweenness"][d])